# FordGoBike_DataVisualization
This is an exploraton of the the FordGoBike dataset from San Francisco in 2018.  

**Objective**: This notebook will explore the dataset and use visualizations to gain insight about trends in rider usage. 